In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = 15
epochs_g = 50
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-Adam-without-pretrained-model/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-Adam-without-pretrained-model/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.SGD(G.parameters(), momentum = 0.9, lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), momentum = 0.9, lr = eta_D, weight_decay = weight_decay_d)

In [7]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 0.69233] [Time:7.37998] 
	[Discriminator][Epochs 2/15] [D epoch loss: 0.69061] [Time:8.97840] 
	[Discriminator][Epochs 3/15] [D epoch loss: 0.68890] [Time:9.52679] 
	[Discriminator][Epochs 4/15] [D epoch loss: 0.68722] [Time:8.67031] 
	[Discriminator][Epochs 5/15] [D epoch loss: 0.68555] [Time:8.67675] 
	[Discriminator][Epochs 6/15] [D epoch loss: 0.68389] [Time:8.59300] 
	[Discriminator][Epochs 7/15] [D epoch loss: 0.68226] [Time:8.39489] 
	[Discriminator][Epochs 8/15] [D epoch loss: 0.68063] [Time:8.37993] 
	[Discriminator][Epochs 9/15] [D epoch loss: 0.67903] [Time:9.35485] 
	[Discriminator][Epochs 10/15] [D epoch loss: 0.67744] [Time:10.28964] 
	[Discriminator][Epochs 11/15] [D epoch loss: 0.67586] [Time:10.82711] 
	[Discriminator][Epochs 12/15] [D epoch loss: 0.67430] [Time:9.64658] 
	[Discriminator][Epochs 13/15] [D epoch loss: 0.67276] [Time:9.90890] 
	[Discriminator][Epochs 14/15] [D epoch loss: 0.67123] [Time:9.93862] 

	[Generator][Epochs 42/50] [G epoch loss: -6.42866] [Time:0.04015]
	[Generator][Epochs 43/50] [G epoch loss: -6.37137] [Time:0.04006]
	[Generator][Epochs 44/50] [G epoch loss: -6.34920] [Time:0.04758]
	[Generator][Epochs 45/50] [G epoch loss: -6.34512] [Time:0.03303]
	[Generator][Epochs 46/50] [G epoch loss: -6.33410] [Time:0.04168]
	[Generator][Epochs 47/50] [G epoch loss: -6.34719] [Time:0.03833]
	[Generator][Epochs 48/50] [G epoch loss: -6.36061] [Time:0.03828]
	[Generator][Epochs 49/50] [G epoch loss: -6.34386] [Time:0.03905]
	[Generator][Epochs 50/50] [G epoch loss: -6.34252] [Time:0.03530]
[Epoch 3/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 0.64921] [Time:9.07834] 
	[Discriminator][Epochs 2/15] [D epoch loss: 0.64796] [Time:9.72105] 
	[Discriminator][Epochs 3/15] [D epoch loss: 0.64672] [Time:10.47153] 
	[Discriminator][Epochs 4/15] [D epoch loss: 0.64548] [Time:10.34908] 
	[Discriminator][Epochs 5/15] [D epoch loss: 0.64426] [Time:10.45820] 
	[Discriminator][Epochs 6/15] 

	[Generator][Epochs 32/50] [G epoch loss: -12.30851] [Time:0.04688]
	[Generator][Epochs 33/50] [G epoch loss: -12.27234] [Time:0.04957]
	[Generator][Epochs 34/50] [G epoch loss: -12.19866] [Time:0.04298]
	[Generator][Epochs 35/50] [G epoch loss: -12.20177] [Time:0.04485]
	[Generator][Epochs 36/50] [G epoch loss: -12.19112] [Time:0.03710]
	[Generator][Epochs 37/50] [G epoch loss: -12.25499] [Time:0.03870]
	[Generator][Epochs 38/50] [G epoch loss: -12.21803] [Time:0.03648]
	[Generator][Epochs 39/50] [G epoch loss: -12.12461] [Time:0.03666]
	[Generator][Epochs 40/50] [G epoch loss: -12.10867] [Time:0.03108]
	[Generator][Epochs 41/50] [G epoch loss: -12.06419] [Time:0.03868]
	[Generator][Epochs 42/50] [G epoch loss: -12.09504] [Time:0.04364]
	[Generator][Epochs 43/50] [G epoch loss: -12.20135] [Time:0.04497]
	[Generator][Epochs 44/50] [G epoch loss: -12.28705] [Time:0.03647]
	[Generator][Epochs 45/50] [G epoch loss: -12.28285] [Time:0.03993]
	[Generator][Epochs 46/50] [G epoch loss: -12.32

	[Generator][Epochs 21/50] [G epoch loss: -17.15008] [Time:0.03710]
	[Generator][Epochs 22/50] [G epoch loss: -17.20111] [Time:0.03763]
	[Generator][Epochs 23/50] [G epoch loss: -17.13589] [Time:0.03807]
	[Generator][Epochs 24/50] [G epoch loss: -17.24580] [Time:0.03829]
	[Generator][Epochs 25/50] [G epoch loss: -17.28540] [Time:0.03800]
	[Generator][Epochs 26/50] [G epoch loss: -17.15790] [Time:0.03782]
	[Generator][Epochs 27/50] [G epoch loss: -17.13205] [Time:0.03853]
	[Generator][Epochs 28/50] [G epoch loss: -17.17962] [Time:0.03906]
	[Generator][Epochs 29/50] [G epoch loss: -17.18797] [Time:0.03855]
	[Generator][Epochs 30/50] [G epoch loss: -17.20431] [Time:0.04010]
	[Generator][Epochs 31/50] [G epoch loss: -17.18440] [Time:0.03874]
	[Generator][Epochs 32/50] [G epoch loss: -17.21585] [Time:0.03679]
	[Generator][Epochs 33/50] [G epoch loss: -17.20398] [Time:0.03675]
	[Generator][Epochs 34/50] [G epoch loss: -17.25471] [Time:0.03399]
	[Generator][Epochs 35/50] [G epoch loss: -17.27

	[Generator][Epochs 11/50] [G epoch loss: -21.53273] [Time:0.02868]
	[Generator][Epochs 12/50] [G epoch loss: -21.48174] [Time:0.04898]
	[Generator][Epochs 13/50] [G epoch loss: -21.48677] [Time:0.03759]
	[Generator][Epochs 14/50] [G epoch loss: -21.55033] [Time:0.03727]
	[Generator][Epochs 15/50] [G epoch loss: -21.57910] [Time:0.03909]
	[Generator][Epochs 16/50] [G epoch loss: -21.72040] [Time:0.03252]
	[Generator][Epochs 17/50] [G epoch loss: -21.72256] [Time:0.03190]
	[Generator][Epochs 18/50] [G epoch loss: -21.71764] [Time:0.03893]
	[Generator][Epochs 19/50] [G epoch loss: -21.52727] [Time:0.03161]
	[Generator][Epochs 20/50] [G epoch loss: -21.46400] [Time:0.03893]
	[Generator][Epochs 21/50] [G epoch loss: -21.52849] [Time:0.03340]
	[Generator][Epochs 22/50] [G epoch loss: -21.49599] [Time:0.04009]
	[Generator][Epochs 23/50] [G epoch loss: -21.45229] [Time:0.04316]
	[Generator][Epochs 24/50] [G epoch loss: -21.60231] [Time:0.04123]
	[Generator][Epochs 25/50] [G epoch loss: -21.52

...

	[Discriminator][Epochs 7/15] [D epoch loss: 0.43963] [Time:9.74657] 
	[Discriminator][Epochs 8/15] [D epoch loss: 0.43956] [Time:9.00969] 
	[Discriminator][Epochs 9/15] [D epoch loss: 0.43949] [Time:9.49981] 
	[Discriminator][Epochs 10/15] [D epoch loss: 0.43943] [Time:9.71367] 
	[Discriminator][Epochs 11/15] [D epoch loss: 0.43933] [Time:10.24365] 
	[Discriminator][Epochs 12/15] [D epoch loss: 0.43926] [Time:10.35813] 
	[Discriminator][Epochs 13/15] [D epoch loss: 0.43920] [Time:10.20015] 
	[Discriminator][Epochs 14/15] [D epoch loss: 0.43913] [Time:10.16003] 
	[Discriminator][Epochs 15/15] [D epoch loss: 0.43907] [Time:10.17947] 
	[Generator][Epochs 1/50] [G epoch loss: -60.20197] [Time:0.04017]
	[Generator][Epochs 2/50] [G epoch loss: -60.22639] [Time:0.03902]
	[Generator][Epochs 3/50] [G epoch loss: -60.02221] [Time:0.03818]
	[Generator][Epochs 4/50] [G epoch loss: -60.18088] [Time:0.03806]
	[Generator][Epochs 5/50] [G epoch loss: -59.96544] [Time:0.03830]
	[Generator][Epochs 6/50

	[Generator][Epochs 47/50] [G epoch loss: -60.17568] [Time:0.03860]
	[Generator][Epochs 48/50] [G epoch loss: -60.03090] [Time:0.03835]
	[Generator][Epochs 49/50] [G epoch loss: -59.97905] [Time:0.03849]
	[Generator][Epochs 50/50] [G epoch loss: -59.90628] [Time:0.03939]
[Epoch 194/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 0.43830] [Time:9.91686] 
	[Discriminator][Epochs 2/15] [D epoch loss: 0.43823] [Time:9.09474] 
	[Discriminator][Epochs 3/15] [D epoch loss: 0.43815] [Time:8.89111] 
	[Discriminator][Epochs 4/15] [D epoch loss: 0.43808] [Time:10.22902] 
	[Discriminator][Epochs 5/15] [D epoch loss: 0.43801] [Time:10.11949] 
	[Discriminator][Epochs 6/15] [D epoch loss: 0.43776] [Time:8.73118] 
	[Discriminator][Epochs 7/15] [D epoch loss: 0.43769] [Time:8.94103] 
	[Discriminator][Epochs 8/15] [D epoch loss: 0.43762] [Time:9.87681] 
	[Discriminator][Epochs 9/15] [D epoch loss: 0.43755] [Time:9.35169] 
	[Discriminator][Epochs 10/15] [D epoch loss: 0.43748] [Time:9.07589] 
	[Discrim

	[Generator][Epochs 37/50] [G epoch loss: -59.97706] [Time:0.03925]
	[Generator][Epochs 38/50] [G epoch loss: -60.11288] [Time:0.03775]
	[Generator][Epochs 39/50] [G epoch loss: -60.21903] [Time:0.03789]
	[Generator][Epochs 40/50] [G epoch loss: -60.32697] [Time:0.03916]
	[Generator][Epochs 41/50] [G epoch loss: -60.11102] [Time:0.03916]
	[Generator][Epochs 42/50] [G epoch loss: -60.02311] [Time:0.03911]
	[Generator][Epochs 43/50] [G epoch loss: -59.92057] [Time:0.03444]
	[Generator][Epochs 44/50] [G epoch loss: -60.11791] [Time:0.03501]
	[Generator][Epochs 45/50] [G epoch loss: -60.21136] [Time:0.04135]
	[Generator][Epochs 46/50] [G epoch loss: -59.95077] [Time:0.03914]
	[Generator][Epochs 47/50] [G epoch loss: -60.09591] [Time:0.03901]
	[Generator][Epochs 48/50] [G epoch loss: -60.08348] [Time:0.03953]
	[Generator][Epochs 49/50] [G epoch loss: -60.05968] [Time:0.02706]
	[Generator][Epochs 50/50] [G epoch loss: -60.31755] [Time:0.02783]
[Epoch 196/200]
	[Discriminator][Epochs 1/15] [D

	[Generator][Epochs 27/50] [G epoch loss: -60.43684] [Time:0.03056]
	[Generator][Epochs 28/50] [G epoch loss: -60.41777] [Time:0.02999]
	[Generator][Epochs 29/50] [G epoch loss: -60.42768] [Time:0.03009]
	[Generator][Epochs 30/50] [G epoch loss: -60.30506] [Time:0.03820]
	[Generator][Epochs 31/50] [G epoch loss: -60.60318] [Time:0.03697]
	[Generator][Epochs 32/50] [G epoch loss: -60.35670] [Time:0.03631]
	[Generator][Epochs 33/50] [G epoch loss: -60.41362] [Time:0.03727]
	[Generator][Epochs 34/50] [G epoch loss: -60.46765] [Time:0.03632]
	[Generator][Epochs 35/50] [G epoch loss: -60.39993] [Time:0.03749]
	[Generator][Epochs 36/50] [G epoch loss: -60.47958] [Time:0.03874]
	[Generator][Epochs 37/50] [G epoch loss: -60.37113] [Time:0.03738]
	[Generator][Epochs 38/50] [G epoch loss: -60.31713] [Time:0.03695]
	[Generator][Epochs 39/50] [G epoch loss: -60.29705] [Time:0.03618]
	[Generator][Epochs 40/50] [G epoch loss: -60.32442] [Time:0.03734]
	[Generator][Epochs 41/50] [G epoch loss: -60.21

	[Generator][Epochs 17/50] [G epoch loss: -60.38107] [Time:0.03852]
	[Generator][Epochs 18/50] [G epoch loss: -60.60098] [Time:0.03846]
	[Generator][Epochs 19/50] [G epoch loss: -60.45885] [Time:0.03943]
	[Generator][Epochs 20/50] [G epoch loss: -60.55626] [Time:0.03929]
	[Generator][Epochs 21/50] [G epoch loss: -60.63538] [Time:0.04029]
	[Generator][Epochs 22/50] [G epoch loss: -60.42302] [Time:0.03807]
	[Generator][Epochs 23/50] [G epoch loss: -60.50846] [Time:0.03972]
	[Generator][Epochs 24/50] [G epoch loss: -60.43226] [Time:0.03962]
	[Generator][Epochs 25/50] [G epoch loss: -60.56160] [Time:0.03810]
	[Generator][Epochs 26/50] [G epoch loss: -60.30622] [Time:0.03998]
	[Generator][Epochs 27/50] [G epoch loss: -60.32438] [Time:0.03954]
	[Generator][Epochs 28/50] [G epoch loss: -60.16313] [Time:0.03689]
	[Generator][Epochs 29/50] [G epoch loss: -60.31748] [Time:0.03926]
	[Generator][Epochs 30/50] [G epoch loss: -60.29684] [Time:0.03943]
	[Generator][Epochs 31/50] [G epoch loss: -60.37